<a href="https://colab.research.google.com/github/gb08/Deep_learning_with_Colab/blob/master/DL_regressor_keras_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

In [0]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
from tensorflow import keras  # tf.keras
import time

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

In [0]:
drive.mount('/content/gdrive')
#!ls gdrive/My\ Drive

model_path = '/content/gdrive/My\ Drive/dl_reg_keras_api'

In [0]:
%matplotlib inline
%load_ext tensorboard

Fetch Dataset

In [0]:

fetch_california_housing = fetch_california_housing()

In [0]:
X = fetch_california_housing.data
Y = fetch_california_housing.target

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.4)

In [0]:
X.shape, X_train.shape, X_val.shape, X_test.shape

In [0]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [0]:
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

## FUNCTIONAL API ##

In [0]:
input_layer = keras.layers.Input(shape=(8,))

In [0]:
hidden1 = keras.layers.Dense(30, activation="relu")(input_layer) 
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)

In [0]:
concat_layer = keras.layers.concatenate(inputs=[input_layer, hidden2]) 

In [0]:
output_layer = keras.layers.Dense(1)(concat_layer)

In [0]:
model = keras.models.Model(inputs=[input_layer], outputs=[output_layer])

In [0]:
model.compile(optimizer=keras.optimizers.SGD(learning_rate=(1e-3)), loss = "mean_squared_error")

In [0]:
model.summary()

In [0]:
history = model.fit(x=X_train_scaled, y=y_train, epochs=10, 
                    validation_data=(X_val_scaled, y_val))

In [0]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()

In [0]:
plot_learning_curves(history)

In [0]:
model.evaluate(x=X_test_scaled, y=y_test)

In [0]:
model.predict(X_test[1:10])

In [0]:
model.save('dl_reg_keras_func_api.h5')

## SUBCLASSING API ##

In [0]:
class MyModel(keras.models.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.hidden1 = keras.layers.Dense(40, activation="relu")
        self.hidden2 = keras.layers.Dense(40, activation="relu")
        self.output_ = keras.layers.Dense(1)

    def call(self, input):
        hidden1 = self.hidden1(input)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input, hidden2])
        output = self.output_(concat)
        return output



In [0]:
model = MyModel()

In [0]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(1e-3))


In [0]:
history = model.fit(X_train_scaled, y_train, epochs=10,
                    validation_data=(X_val_scaled, y_val))

In [0]:
model.summary()


In [0]:
model.evaluate(X_test_scaled, y_test)


In [0]:
model.save('dl_reg_keras_succl_api', save_format='tf')

In [0]:
X_train_scaled_A = X_train_scaled[:, :5]
X_train_scaled_B = X_train_scaled[:, 2:]
X_valid_scaled_A = X_val_scaled[:, :5]
X_valid_scaled_B = X_val_scaled[:, 2:]
X_test_scaled_A = X_test_scaled[:, :5]
X_test_scaled_B = X_test_scaled[:, 2:]

In [0]:
input_A = keras.layers.Input(shape=X_train_scaled_A.shape[1:])
input_B = keras.layers.Input(shape=X_train_scaled_B.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1)(concat)
aux_output = keras.layers.Dense(1)(hidden2)

In [0]:
model = keras.models.Model(inputs=[input_A, input_B],
                           outputs=[output, aux_output])


In [0]:
model.compile(loss=["mean_squared_error", "mean_squared_error"], loss_weights=[0.9, 0.1],
              optimizer=keras.optimizers.SGD(1e-3))


In [0]:
model.summary()


In [0]:
history = model.fit([X_train_scaled_A, X_train_scaled_B], [y_train, y_train], epochs=10,
                    validation_data=([X_valid_scaled_A, X_valid_scaled_B], [y_val, y_val]))


In [0]:
model.evaluate([X_test_scaled_A, X_test_scaled_B], [y_test, y_test])


In [0]:
y_pred, y_pred_aux = model.predict([X_test_scaled_A, X_test_scaled_B])
